# DEM Differencing from IFSAR with Nuth and Kaab coregistration

In [ ]:
import os
import re
import xdem
from datetime import datetime
import matplotlib.pyplot as plt
import geoutils as gu
import numpy as np
import pyproj
import pandas as pd
import glob
from tqdm.auto import tqdm

# root directory to all data
data_path = '/Users/raineyaberle/Research/PhD/Hubbard/'
# path for saving figures
figures_out_path = os.path.join(data_path, 'figures')

## Load glacier boundaries and IFSAR DEM over Hubbard

In [ ]:
# -----Load glacier boundaries
aoi_fn = os.path.join(data_path, 'glacier_boundaries', 'Hubbard_boundaries.shp')
# Create geoutils vector obect from AOI for calculating NMAD in AOI
aoi_gu = gu.Vector(aoi_fn)

# -----Load IFSAR DEM
ifsar_fn = os.path.join(data_path, 'surface_elevations', 'ifsar_hubbardDEM.tif')
# load ifsar dem in AK albers projection
ifsar = xdem.DEM(ifsar_fn)
# tell xdem what vertical projection it's in
ifsar.set_vcrs(pyproj.CRS("EPSG:5703"))
# set vcrs from geoid to ellipsoid
ifsar.to_vcrs("Ellipsoid")

# -----Reproject AOI to IFSAR CRS
aoi_gu = aoi_gu.reproject(ifsar)

# -----Plot
fig, ax = plt.subplots(1, 1, figsize=(6,6))
dem_im = ax.imshow(ifsar.data, cmap="terrain", clim=(0, 2.5e3),
                   extent=(ifsar.bounds.left, ifsar.bounds.right, 
                           ifsar.bounds.bottom, ifsar.bounds.top))
fig.colorbar(dem_im, ax=ax, shrink=0.5, label='Elevation [m]')
aoi_gu.show(ax=ax, facecolor='None', edgecolor='k')
ax.set_xlabel('Easting [m]')
ax.set_ylabel('Northing [m]')
plt.show()

## Coregister and difference ArcticDEM strips and IFSAR

In [ ]:
# -----Grab ArcticDEM Strip file names
dem_fns = sorted(glob.glob(os.path.join(data_path, 'surface_elevations', 'ArcticDEM', '*.tif')))
print('Number of files = ', len(dem_fns))
[os.path.basename(x) for x in dem_fns]

In [ ]:
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})

# -----Iterate over DEM file names
for dem_fn in tqdm(dem_fns):

    # Grab DEM date from file name
    dem_date = os.path.basename(dem_fn)[18:22] + '-' + os.path.basename(dem_fn)[22:24]  + '-' + os.path.basename(dem_fn)[24:26]

    # Check if coregistered DEM already exists in file
    # dem_out_fn = dem_fn[0:-4] + '_coreg.tif'
    # if os.path.exists(dem_out_fn):
    
    # load coregistered DEM
    aligned_dem = xdem.DEM(dem_fn)
    print('Coregistered DEM loaded from file')
        
    # else:
    
    #     # load DEM as xdem.DEM
    #     dem = xdem.DEM(dem_fn)
    #     # account for no data values
    #     dem.data.data[(dem.data.data==-9999) | (dem.data.data==0) | (dem.data.data==-99999)] = np.nan
    
    #     # Reproject DEM to IFSAR CRS
    #     dem = dem.reproject(ifsar)
    
    #     # Coregister DEM to IFSAR DEM
    #     nuth_kaab = xdem.coreg.NuthKaab()
    #     try:
    #         nuth_kaab.fit(ifsar, dem) # Fit the data to a suitable x/y/z offset
    #         aligned_dem = nuth_kaab.apply(dem) # Apply the transformation to the data
    #     except Exception as e:
    #         print(e)
    #         continue
    
    #     # Save coregistered DEM to file
    #     aligned_dem.save(dem_out_fn)
    #     print('Coregistered DEM saved to file: ', dem_out_fn)
    
    # Calculate the elevation difference (ArcticDEM - IFSAR)
    aligned_diff = aligned_dem - ifsar

    # Calculate NMAD before and after coregistration
    # inlier_mask = ~aoi_gu.create_mask(dem) # Create a mask of points in the AOI
    # nmad_pre = xdem.spatialstats.nmad(dem[inlier_mask])
    # nmad_post = xdem.spatialstats.nmad(aligned_dem[inlier_mask])
    
    # Plot 
    vmin, vmax = -20, 20
    xmin, xmax = ifsar.bounds.left-500, ifsar.bounds.right+500
    ymin, ymax = ifsar.bounds.bottom-500, ifsar.bounds.top+500
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    # Coregistered DEM
    dem_im = ax[0].imshow(aligned_dem.data, cmap="terrain", vmin=0, vmax=2.5e3,
                          extent=(ifsar.bounds.left, ifsar.bounds.right, 
                                  ifsar.bounds.bottom, ifsar.bounds.top))
    fig.colorbar(dem_im, shrink=0.5, label='Elevation [m]')
    aoi_gu.reproject(aligned_dem).show(ax=ax[0], facecolor='None', edgecolor='grey')
    # Coregistered DEM difference
    # ax[1].set_title(f"NMAD={nmad_post:.1f} m")
    diff_im = ax[1].imshow(aligned_diff.data, cmap="coolwarm_r", vmin=vmin, vmax=vmax,
                           extent=(ifsar.bounds.left, ifsar.bounds.right, 
                                   ifsar.bounds.bottom, ifsar.bounds.top))
    aoi_gu.reproject(aligned_diff).show(ax=ax[1], facecolor='None', edgecolor='grey')
    for axis in ax:
        axis.set_xlim(xmin, xmax)
        axis.set_ylim(ymin, ymax)
        axis.set_xlabel('Easting [m]')
        axis.set_ylabel('Northing [m]')
    cbar = fig.colorbar(diff_im, shrink=0.5, ax=ax[1], label='Elevation change [m]')
    # Add annotations to legend
    fig.canvas.draw()
    bbox = cbar.ax.get_yticklabels()[-1].get_window_extent()
    labx, _ = cbar.ax.transAxes.inverted().transform([bbox.x1,0]) 
    if int(dem_date[0:4]) > 2012:
        ax[1].annotate('Thinning', xy=(-1, -0.1), xycoords=cbar.ax.transAxes,
                   fontweight='bold', color=plt.cm.coolwarm_r(0))
        ax[1].annotate('Thickening', xy=(-1, 1.1), xycoords=cbar.ax.transAxes,
                       fontweight='bold', color=plt.cm.coolwarm_r(10000))
    elif int(dem_date[0:4]) < 2010:
        ax[1].annotate('Thickening', xy=(-1, -0.1), xycoords=cbar.ax.transAxes,
                   fontweight='bold', color=plt.cm.coolwarm_r(0))
        ax[1].annotate('Thinning', xy=(-1, 1.1), xycoords=cbar.ax.transAxes,
                       fontweight='bold', color=plt.cm.coolwarm_r(10000))
    fig.suptitle(dem_date + '\nArcticDEM-IFSAR')
    plt.tight_layout()
    plt.show()

    # Save figure
    fig_fn = os.path.join(figures_out_path, 'elevation_diff_' + dem_date + '_ArcticDEM-IFSAR.png')
    fig.savefig(fig_fn, dpi=200, bbox_inches='tight')
    print('figure saved to file: ', fig_fn)
    print(' ')


## Coregister and difference OIB Lidar and IFSAR

In [ ]:
# -----Grab OIB Lidar file names
dem_fns = sorted(glob.glob(os.path.join(data_path, 'surface_elevations', 'OIB_lidar', '*10m.tif')))
print('Number of files = ', len(dem_fns))
[os.path.basename(x) for x in dem_fns]

In [ ]:
import datetime
import rasterio as rio
plt.rcParams.update({'font.sans-serif':'Arial', 'font.size':12})

# -----Iterate over DEM file names
for dem_fn in tqdm(dem_fns):

    # Grab DEM date from file name
    if 'Hubbard.' in dem_fn:
        dem_year = dem_fn.split('Hubbard.')[1][0:4]
        dem_julian_day = dem_fn.split('Hubbard.' + dem_year + '.')[1][0:3]
        dem_date = str(datetime.datetime.strptime(dem_year + dem_julian_day, '%Y%j').date())
    elif 'Valerie.' in dem_fn:
        dem_year = dem_fn.split('Valerie.')[1][0:4]
        dem_julian_day = dem_fn.split('Valerie.' + dem_year + '.')[1][0:3]
        dem_date = str(datetime.datetime.strptime(dem_year + dem_julian_day, '%Y%j').date())
    elif 'ILAKS' in dem_fn:
        dem_year = dem_fn.split('ILAKS1B_')[1][0:4]
        dem_julian_day = dem_fn.split('ILAKS1B_' + dem_year + '_')[1][0:3]
        dem_date = str(datetime.datetime.strptime(dem_year + dem_julian_day, '%Y%j').date())

    print(dem_date)
    
    # Check if coregistered DEM already exists in file
    dem_out_fn = dem_fn[0:-4] + '_coreg.tif'
    if os.path.exists(dem_out_fn):
        # load coregistered DEM
        aligned_dem = xdem.DEM(dem_out_fn)
        print('Coregistered DEM loaded from file')
        
    else:

        # load DEM using rasterio to extract just the first band (mean point elevations)
        with rio.open(dem_fn) as src:
            array = src.read(1)
            transform = src.transform
            crs = src.crs
        # load DEM as xdem.DEM
        dem = xdem.DEM.from_array(array, transform=transform, crs=crs)
        # account for no data values
        dem.data.data[(dem.data.data==-9999) | (dem.data.data==0) | (dem.data.data==-99999)] = np.nan
    
        # Reproject DEM to IFSAR CRS
        dem = dem.reproject(ifsar)
    
        # Coregister DEM to IFSAR DEM
        nuth_kaab = xdem.coreg.NuthKaab()
        try:
            nuth_kaab.fit(ifsar, dem) # Fit the data to a suitable x/y/z offset
            aligned_dem = nuth_kaab.apply(dem) # Apply the transformation to the data
        except Exception as e:
            print(e)
            continue
    
        # Save coregistered DEM to file
        aligned_dem.save(dem_out_fn)
        print('Coregistered DEM saved to file: ', dem_out_fn)
    
    # Calculate the elevation difference (ArcticDEM - IFSAR)
    aligned_diff = aligned_dem - ifsar

    # Calculate NMAD before and after coregistration
    # inlier_mask = ~aoi_gu.create_mask(dem) # Create a mask of points in the AOI
    # nmad_pre = xdem.spatialstats.nmad(dem[inlier_mask])
    # nmad_post = xdem.spatialstats.nmad(aligned_dem[inlier_mask])
    
    # Plot 
    vmin, vmax = -20, 20
    xmin, xmax = ifsar.bounds.left-500, ifsar.bounds.right+500
    ymin, ymax = ifsar.bounds.bottom-500, ifsar.bounds.top+500
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    # Coregistered DEM
    dem_im = ax[0].imshow(aligned_dem.data, cmap="terrain", vmin=0, vmax=2.5e3,
                          extent=(ifsar.bounds.left, ifsar.bounds.right, 
                                  ifsar.bounds.bottom, ifsar.bounds.top))
    fig.colorbar(dem_im, shrink=0.5, label='Elevation [m]')
    aoi_gu.reproject(aligned_dem).show(ax=ax[0], facecolor='None', edgecolor='grey')
    # Coregistered DEM difference
    # ax[1].set_title(f"NMAD={nmad_post:.1f} m")
    diff_im = ax[1].imshow(aligned_diff.data, cmap="coolwarm_r", vmin=vmin, vmax=vmax,
                           extent=(ifsar.bounds.left, ifsar.bounds.right, 
                                   ifsar.bounds.bottom, ifsar.bounds.top))
    aoi_gu.reproject(aligned_diff).show(ax=ax[1], facecolor='None', edgecolor='grey')
    for axis in ax:
        axis.set_xlim(xmin, xmax)
        axis.set_ylim(ymin, ymax)
        axis.set_xlabel('Easting [m]')
        axis.set_ylabel('Northing [m]')
    cbar = fig.colorbar(diff_im, shrink=0.5, ax=ax[1], label='Elevation change [m]')
    # Add annotations to legend
    fig.canvas.draw()
    bbox = cbar.ax.get_yticklabels()[-1].get_window_extent()
    labx, _ = cbar.ax.transAxes.inverted().transform([bbox.x1,0]) 
    if int(dem_date[0:4]) > 2012:
        ax[1].annotate('Thinning', xy=(-1, -0.1), xycoords=cbar.ax.transAxes,
                   fontweight='bold', color=plt.cm.coolwarm_r(0))
        ax[1].annotate('Thickening', xy=(-1, 1.1), xycoords=cbar.ax.transAxes,
                       fontweight='bold', color=plt.cm.coolwarm_r(10000))
    elif int(dem_date[0:4]) < 2010:
        ax[1].annotate('Thickening', xy=(-1, -0.1), xycoords=cbar.ax.transAxes,
                   fontweight='bold', color=plt.cm.coolwarm_r(0))
        ax[1].annotate('Thinning', xy=(-1, 1.1), xycoords=cbar.ax.transAxes,
                       fontweight='bold', color=plt.cm.coolwarm_r(10000))
    fig.suptitle(dem_date + '\nOIB-IFSAR')
    plt.tight_layout()
    plt.show()

    # Save figure
    fig_fn = os.path.join(figures_out_path, 'elevation_diff_' + dem_date + '_OIB-IFSAR.png')
    fig.savefig(fig_fn, dpi=200, bbox_inches='tight')
    print('figure saved to file: ', fig_fn)
    print(' ')


## Create a gif of all output figures using ImageMagick

In [ ]:
import subprocess
os.chdir(figures_out_path)
cmd = "convert -delay 100 elevation_diff*.png elevation_diff.gif"
output = subprocess.run(cmd, shell=True, capture_output=True)
print(output)

## Create movie of all output figures using [`ffmpeg`](https://ffmpeg.org/)

In [ ]:
import subprocess
os.chdir(figures_out_path)
cmd = "ffmpeg -framerate 1000 -pattern_type glob -i 'elevation_diff*.png' -c:v libx264 elevation_diff.mp4"
output = subprocess.run(cmd, shell=True, capture_output=True)
print(output)